In [1]:
# load packages
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
tf.__version__

'1.12.0'

# Load Data

In [2]:
data = pd.read_csv("data/iris-dataset.csv")
data.shape, data.columns

((150, 6),
 Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
        'Species'],
       dtype='object'))

In [3]:
target_values = data.Species.unique()
target_values

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [4]:
# create mapping string -> numeric value
target_values_mapping = dict()
i = 0
for tar in target_values:
    if tar not in target_values_mapping:
        target_values_mapping[tar] = i
        i += 1
    else:
        continue

In [5]:
target_values_mapping

{'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}

In [6]:
# convert string value to numeric values
data.Species = data.Species.apply(lambda x: target_values_mapping[x])
data.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,0
1,2,4.9,3.0,1.4,0.2,0
2,3,4.7,3.2,1.3,0.2,0
3,4,4.6,3.1,1.5,0.2,0
4,5,5.0,3.6,1.4,0.2,0


In [7]:
data.Species.unique()

array([0, 1, 2])

In [8]:
features = np.array(data.iloc[:, 1:-1]) # load features into a numpy array
labels = np.array(data.iloc[:, [-1]]) # load target values into a numpy array

labels.shape, features.shape

((150, 1), (150, 4))

In [9]:
features[:3], labels[:3]

(array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2]]), array([[0],
        [0],
        [0]]))

In [10]:
labels = tf.keras.utils.to_categorical(labels, num_classes=3)

In [11]:
# split data into train and validate
val_split_size = 0.2
random_indices = np.random.rand(len(features)) < val_split_size

# split data
train_x = features[~random_indices]
val_x = features[random_indices]
train_y = labels[~random_indices]
val_y = labels[random_indices]

In [12]:
train_x.shape, train_y.shape, val_x.shape, val_y.shape

((106, 4), (106, 3), (44, 4), (44, 3))

# Multilayer Perceptron Model

In [13]:
num_features = train_x.shape[1]
num_classes = train_y.shape[1]

In [42]:
num_features, num_classes

(4, 3)

In [43]:
# placeholders
x = tf.placeholder(dtype=tf.float32, shape=(None, num_features), name="feature_x")
y = tf.placeholder(dtype=tf.float32, shape=(None, num_classes), name="target_y")

In [44]:
# define baias for layers
bias = tf.Variable([[1.0]], dtype=tf.float32, name="layer_bias")

In [45]:
# configure layer 1
l1_nodes = 10
w1 = tf.Variable(tf.random_normal(shape=(l1_nodes, num_features), mean=0.0, stddev=1.0, dtype=tf.float32, name="w1"))
l1_output = tf.nn.relu(tf.add(tf.matmul(x, tf.transpose(w1)), bias))

In [46]:
# configure second hidden layer
l2_nodes = 32
w2 = tf.Variable(tf.random_normal(shape=(l2_nodes, l1_nodes), mean=0.0, stddev=1.0, dtype=tf.float32, name="w2"))
l2_output = tf.nn.relu(tf.add(tf.matmul(l1_output, tf.transpose(w2)), bias))

In [47]:
# output layer
w3 = tf.Variable(tf.random_normal(shape=(num_classes, l2_nodes), mean=0.0, stddev=1.0, dtype=tf.float32, name="w3"))
l3_output = tf.nn.softmax(tf.add(tf.matmul(l2_output, tf.transpose(w3)), bias))

In [48]:
# define loss function
# loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=l3_output), name="loss")
loss_function = tf.reduce_mean(-tf.reduce_sum(y*tf.log(l3_output + 0.0001)))

In [49]:
trainer = tf.train.RMSPropOptimizer(learning_rate = 0.01).minimize(loss_function)

In [50]:
# run the model
epochs = 1000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epochs):
        sess.run(trainer, feed_dict={x:train_x, y:train_y})
        if i % 100 == 0:
            train_loss = sess.run(loss_function, feed_dict={x:train_x, y:train_y})
            print("Step:", i, "Train Loss:", train_loss)
    val_loss = sess.run(loss_function, feed_dict={x:val_x, y:val_y})
    predicted = sess.run(l3_output, feed_dict={x:val_x, y:val_y})
    print("Validation Loss:", val_loss)

Step: 0 Train Loss: 369.03073
Step: 100 Train Loss: 303.934
Step: 200 Train Loss: 303.934
Step: 300 Train Loss: 303.93393
Step: 400 Train Loss: 62.686295
Step: 500 Train Loss: 43.670845
Step: 600 Train Loss: 27.073212
Step: 700 Train Loss: 16.737186
Step: 800 Train Loss: 10.015486
Step: 900 Train Loss: 8.215415
Validation Loss: 1.3870778


In [51]:
predicted_labels = list()
for res in predicted:
    predicted_labels.append(np.argmax(res))

In [52]:
len(predicted_labels)

44

In [53]:
original_labels = list()
for lab in val_y:
    original_labels.append(np.argmax(lab))

In [54]:
len(original_labels)

44

In [55]:
from sklearn.metrics import accuracy_score
# accurayc score
accuracy_score(predicted_labels, original_labels)

1.0